lunana  
last update 2022 05 10  
ゆっくりしていってね！  

version 3 learning_bert  
version 4 cpc-context

# Trends  

* iterate like a grandmaster (method with deverta-v3)  
https://www.kaggle.com/code/jhoward/iterate-like-a-grandmaster 

## Add test  
* USPPPM Nakama's Deberta-v3-large train add test  
  seed 42 to 41
* USPPPM Nakama's Deberta-v3-large infer add test  
  LB=0.836  
  publick notebook to private
  

# Data list  
* [**train.csv**](#train.csv)  
* [**test.csv**](#test.csv)  
* [**sample_submission.csv**](#sample_submission.csv)  
* [**learning_BERT**](#learning_BERT)  
* [**CPC_context**](#CPC_context)

**霊夢：今日も自然言語処理のコンペだね。  
魔理沙：まずは概要を読んでみよう。**

**Reimu: It's a natural language processing competition today.  
Marisa: First, let's read the overview.**

発明から派生した大規模なテキストベースのデータセットから意味を抽出できますか？これがあなたのチャンスです。

米国特許商標庁（USPTO）は、オープンデータポータルを通じて、科学、技術、および商業情報の最大のリポジトリの1つを提供しています。特許は、新しく有用な発明の公開と引き換えに付与される知的財産の一形態です。特許は付与前に徹底的な審査プロセスを経ており、米国のイノベーションの歴史は2世紀以上にわたって1,100万件の特許であるため、米国の特許アーカイブは、データ量、品質、多様性のまれな組み合わせとして存在します。

「USPTOは、特許を付与し、商標を登録し、世界中で知的財産を促進することにより、眠ることのないアメリカのイノベーションマシンにサービスを提供しています。USPTOは、電球から量子コンピューターに至るまで、200年以上に及ぶ人間の創意工夫を世界と共有しています。USPTOデータセットは、データサイエンスコミュニティの創造性と相まって、科学と社会全体の進歩に役立つAIおよびMLモデルを強化する無限の可能性を秘めています。」

—USPTO最高情報責任者JamieHolcombe

このコンテストでは、新しい意味的類似性データセットでモデルをトレーニングし、特許文書のキーフレーズを照合して関連情報を抽出します。フレーズ間の意味的類似性を判断することは、発明が以前に説明されているかどうかを判断するための特許検索および審査プロセス中に非常に重要です。たとえば、ある発明が「テレビセット」を主張し、以前の刊行物が「テレビセット」を説明している場合、モデルはこれらが同じであることを理想的に認識し、弁理士または審査官が関連文書を検索するのを支援します。これは言い換えの識別を超えています。ある発明が「強い材料」を主張し、別の発明が「鋼」を使用する場合、それも一致する可能性があります。「強い素材」と見なされるもの ドメインごとに異なります（あるドメインではスチール、別のドメインではリップストップファブリックの場合がありますが、パラシュートをスチールで作成することは望ましくありません）。これらの状況を明確にするための追加機能として、技術ドメインのコンテキストとして共同特許分類が含まれています。

コンテキスト情報を抽出するためにフレーズに一致するモデルを構築し、それによって特許コミュニティが数百万の特許文書間の点を結ぶのを支援できますか？  

Can you extract meaning from a large, text-based dataset derived from inventions? Here's your chance to do so.

The U.S. Patent and Trademark Office (USPTO) offers one of the largest repositories of scientific, technical, and commercial information in the world through its Open Data Portal. Patents are a form of intellectual property granted in exchange for the public disclosure of new and useful inventions. Because patents undergo an intensive vetting process prior to grant, and because the history of U.S. innovation spans over two centuries and 11 million patents, the U.S. patent archives stand as a rare combination of data volume, quality, and diversity.

“The USPTO serves an American innovation machine that never sleeps by granting patents, registering trademarks, and promoting intellectual property around the globe. The USPTO shares over 200 years' worth of human ingenuity with the world, from lightbulbs to quantum computers. Combined with creativity from the data science community, USPTO datasets carry unbounded potential to empower AI and ML models that will benefit the progress of science and society at large.”

— USPTO Chief Information Officer Jamie Holcombe

In this competition, you will train your models on a novel semantic similarity dataset to extract relevant information by matching key phrases in patent documents. Determining the semantic similarity between phrases is critically important during the patent search and examination process to determine if an invention has been described before. For example, if one invention claims "television set" and a prior publication describes "TV set", a model would ideally recognize these are the same and assist a patent attorney or examiner in retrieving relevant documents. This extends beyond paraphrase identification; if one invention claims a "strong material" and another uses "steel", that may also be a match. What counts as a "strong material" varies per domain (it may be steel in one domain and ripstop fabric in another, but you wouldn't want your parachute made of steel). We have included the Cooperative Patent Classification as the technical domain context as an additional feature to help you disambiguate these situations.

Can you build a model to match phrases in order to extract contextual information, thereby helping the patent community connect the dots between millions of patent documents?

# train.csv

**魔理沙：次はデータを見てみよう**

train.csv - the training set, containing phrases, contexts, and their similarity scores

* id - a unique identifier for a pair of phrases  
* anchor - the first phrase  
* target - the second phrase  
* context - the CPC classification (version 2021.05), which indicates the subject within which the similarity is to be scored  
* score - the similarity. This is sourced from a combination of one or more manual expert ratings.

Score meanings  
The scores are in the 0-1 range with increments of 0.25 with the following meanings:

* 1.0 - Very close match. This is typically an exact match except possibly for differences in conjugation, quantity (e.g. singular vs. plural), and addition or removal of stopwords (e.g. “the”, “and”, “or”).
* 0.75 - Close synonym, e.g. “mobile phone” vs. “cellphone”. This also includes abbreviations, e.g. "TCP" -> "transmission control protocol".
* 0.5 - Synonyms which don’t have the same meaning (same function, same properties). This includes broad-narrow (hyponym) and narrow-broad (hypernym) matches.
* 0.25 - Somewhat related, e.g. the two phrases are in the same high level domain but are not synonyms. This also includes antonyms.
* 0.0 - Unrelated.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt

In [ ]:
df_train = pd.read_csv("../input/us-patent-phrase-to-phrase-matching/train.csv")
df_train.head()

In [ ]:
len(df_train)

In [ ]:
con_uni=df_train['context'].unique()
con_uni=np.sort(con_uni)
con_uni

**霊夢:アルファベットは何を表してるの？  
魔理沙:以下のURLから詳しく見れるよ。**  

**Reimu: What does the alphabet represent?  
Marisa: You can see it in detail from the following URL.**  

https://en.wikipedia.org/wiki/Cooperative_Patent_Classification

A: Human Necessities(人間の必需品)  
B: Operations and Transport(運用と輸送)  
C: Chemistry and Metallurgy(化学および冶金学)  
D: Textiles(テキスタイル)  
E: Fixed Constructions(固定構造)  
F: Mechanical Engineering(機械工学)  
G: Physics(物理学)  
H: Electricity(電気)  
Y: Emerging Cross-Sectional Technologies(新しい横断的テクノロジー)  

![https://3.bp.blogspot.com/-42ddByrXRig/VZ-W1-dVteI/AAAAAAAAvTU/SOdvlTbjoz0/s300/woman_question.png](https://3.bp.blogspot.com/-42ddByrXRig/VZ-W1-dVteI/AAAAAAAAvTU/SOdvlTbjoz0/s300/woman_question.png)

**霊夢:アルファベットでグループ分けしてグラフにしてみよう。**  

**Reimu: Let's group them by alphabet and make a graph.**

![https://4.bp.blogspot.com/-7usFwCGIGWY/WSa84uAcZVI/AAAAAAABEiI/C_HnN-KaIq096NBmvshZZ4t-QhZkjkxXwCLcB/s500/whiteboard_down_woman2.png](https://4.bp.blogspot.com/-7usFwCGIGWY/WSa84uAcZVI/AAAAAAABEiI/C_HnN-KaIq096NBmvshZZ4t-QhZkjkxXwCLcB/s500/whiteboard_down_woman2.png)

In [ ]:
df_train['group']=df_train['context'].apply(lambda x: x[0])
df_train.head()

In [ ]:
df_train['group'].unique()

In [ ]:
label=['A','B','C','D','E','F','G','H']
plt.figure(figsize=(15, 8))
plt.rcParams["font.size"] = 18
plt.bar(df_train["group"].value_counts().sort_index().index,df_train["group"].value_counts().sort_index(),tick_label =label,color="#f4a460" )

**魔理沙:次はscoreのグラフを書いてみよう。**  

**Marisa: Next, let's draw a score graph.**

In [ ]:
plt.figure(figsize=(10, 8))
plt.rcParams["font.size"] = 18
plt.bar(df_train["score"].value_counts().sort_values(ascending=True).index,df_train["score"].value_counts().sort_values(ascending=True),tick_label = df_train["score"].value_counts().sort_values(ascending=True).index,width=0.2,color="#dda0dd")

# test.csv

**魔理沙：次はテストを見てみよう**

test.csv - the test set set, identical in structure to the training set but without the score

In [ ]:
df_test = pd.read_csv("../input/us-patent-phrase-to-phrase-matching/test.csv")
df_test.head()

In [ ]:
len(df_test)

In [ ]:
con_uni=df_test['context'].unique()
con_uni=np.sort(con_uni)
con_uni

In [ ]:
len(con_uni)

**霊夢:テストデータのanchorはtrainデータに同じものあるのかな？**  
**Reimu: Is the anchor of the test data the same as the train data?**

In [ ]:
anchor_list=df_test['anchor'].unique()
anchor_list=np.sort(anchor_list)
anchor_list

In [ ]:
len(anchor_list)

In [ ]:
df=df_train[df_train['anchor'].isin(anchor_list)]
df

In [ ]:
df['anchor'].unique()

In [ ]:
len(df['anchor'].unique())

**魔理沙:テストデータのanchorはtrainデータにあるみたいだな。  
霊夢:targetはどうだろう？  
魔理沙:同じようにして調べてみよう。**  

**Marisa: It seems that the anchor of the test data is in the train data.  
Reimu: What about target?  
Marisa: Let's find out in the same way.**  

In [ ]:
target_list=df_test['target'].unique()
target_list=np.sort(target_list)
target_list

In [ ]:
len(target_list)

In [ ]:
df2=df_train[df_train['target'].isin(target_list)]
df2.head()

In [ ]:
df2['target'].unique()

In [ ]:
len(df2['target'].unique())

**霊夢:あれ？targetもtrainデータに全てあるぞ。  
魔理沙:もしかして、test.csvはtrain.csvからいくつかのデータを引っ張ってきただけなのでは？  
霊夢:idも見てみよう。**  

**Reimu:That? The target is also in the train data.  
Marisa:Maybe test.csv just pulled some data from train.csv?  
Reimu:Let's also look at the id.**

In [ ]:
id_list=df_test['id'].unique()
id_list=np.sort(id_list)
id_list

In [ ]:
len(id_list)

In [ ]:
df3=df_train[df_train['id'].isin(id_list)]
df3.head()

In [ ]:
df3['id'].unique()

In [ ]:
len(df3['id'].unique())

**魔理沙:予想的中。test.csvは全て、同じものがtrain.csvにある。  
霊夢:それは別にいいんだが、実際のデータはどうなんだろう？trainデータにはないcontextがあったりするのかな？  
魔理沙:さあね。** 

**Marisa: After all. All test.csv has the same thing in train.csv.  
Reimu: That's fine, but what about the actual data? Is there a context that is not in the train data?  
Marisa: I don't know.**

# sample_submission.csv

**魔理沙：最後にsample_submissionを見てみよう**  

**Marisa: Finally, let's take a look at sample_submission.**

sample_submission.csv - a sample submission file in the correct format

In [ ]:
sample_submission = pd.read_csv("../input/us-patent-phrase-to-phrase-matching/sample_submission.csv")
sample_submission.head()

# learning_BERT

自然言語処理は、Transformerを使います。多くのライブラリがHuggingFaceに登録されています。  
HuggingFace  
https://huggingface.co/  
BERTとは、Bidirectional Encoder Representations from Transformers の略で、「Transformerによる双方向のエンコード表現」と訳され、2018年10月にGoogleのJacob Devlinらの論文で発表された自然言語処理モデルです。  
日本語解説  
https://ledge.ai/bert/  
今回の文章の類似の前に簡単に質疑応答で試してみます。

##  Similar to sentences

文章類似度は、albertを使うと便利です。tokenizerで変換して、モデルで文章ベクトルを算出し(embeddings)、cosineで類似度を計算します。1が一番似ています。

## English version

In [ ]:
from transformers import AlbertTokenizer,AlbertForPreTraining
import torch

model_name= 'albert-base-v2'
tokenizer = AlbertTokenizer.from_pretrained(model_name)
model = AlbertForPreTraining.from_pretrained(model_name)
 
input_ids = torch.tensor(tokenizer.encode("I am a god", add_special_tokens=True)).unsqueeze(0)  # Batch size 1
outputs = model(input_ids)
prediction_logits = outputs.prediction_logits

In [ ]:
input_ids_A = torch.tensor(tokenizer.encode("I am a god.", add_special_tokens=True)).unsqueeze(0) 
embeddings_A = torch.mean(model(input_ids_A,output_hidden_states=True).hidden_states[12],1)
input_ids_B = torch.tensor(tokenizer.encode("I am a cat.", add_special_tokens=True)).unsqueeze(0)  
embeddings_B = torch.mean(model(input_ids_B,output_hidden_states=True).hidden_states[12],1)
input_ids_C = torch.tensor(tokenizer.encode("Luck is also out of your ability ・ While you are saying such things, your ability is not enough.", add_special_tokens=True)).unsqueeze(0)  
embeddings_C = torch.mean(model(input_ids_C,output_hidden_states=True).hidden_states[12],1)
print(torch.cosine_similarity(embeddings_A,embeddings_B,dim=1))
print(torch.cosine_similarity(embeddings_A,embeddings_C,dim=1))
print(torch.cosine_similarity(embeddings_B,embeddings_C,dim=1))

## Japanese version

In [ ]:
model_name= 'ALINEAR/albert-japanese-v2'
tokenizer = AlbertTokenizer.from_pretrained(model_name)
model = AlbertForPreTraining.from_pretrained(model_name)
 
input_ids = torch.tensor(tokenizer.encode("吾輩は神である。", add_special_tokens=True)).unsqueeze(0)  # Batch size 1
outputs = model(input_ids)
prediction_logits = outputs.prediction_logits

In [ ]:
input_ids_A = torch.tensor(tokenizer.encode("吾輩は神である。", add_special_tokens=True)).unsqueeze(0) 
embeddings_A = torch.mean(model(input_ids_A,output_hidden_states=True).hidden_states[12],1)
input_ids_B = torch.tensor(tokenizer.encode("吾輩は猫である。", add_special_tokens=True)).unsqueeze(0)  
embeddings_B = torch.mean(model(input_ids_B,output_hidden_states=True).hidden_states[12],1)
input_ids_C = torch.tensor(tokenizer.encode("運も実力のうち・そんなことを言っている間は実力が足りない。", add_special_tokens=True)).unsqueeze(0)  
embeddings_C = torch.mean(model(input_ids_C,output_hidden_states=True).hidden_states[12],1)
print(torch.cosine_similarity(embeddings_A,embeddings_B,dim=1))
print(torch.cosine_similarity(embeddings_A,embeddings_C,dim=1))
print(torch.cosine_similarity(embeddings_B,embeddings_C,dim=1))

# CPC_context

cpc_contextは、cpc-dataから作ります。これをcontextにして、debertaで推定に使用します。  
cpc-dataはタブ区切りのtsv形式です。nakama氏のやり方を変更します。

In [ ]:
import os
import re

Nakama's code

In [ ]:
# ====================================================
# CPC Data
# ====================================================
def get_cpc_texts():
    contexts = []
    pattern = '[A-Z]\d+'
    for file_name in os.listdir('../input/cpc-data/CPCSchemeXML202105'):
        result = re.findall(pattern, file_name)
        if result:
            contexts.append(result)
    contexts = sorted(set(sum(contexts, [])))
    results = {}
    for cpc in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'Y']:
        with open(f'../input/cpc-data/CPCTitleList202202/cpc-section-{cpc}_20220201.txt') as f:
            s = f.read()
        pattern = f'{cpc}\t\t.+'
        result = re.findall(pattern, s)
        cpc_result = result[0].lstrip(pattern)
        for context in [c for c in contexts if c[0] == cpc]:
            pattern = f'{context}\t\t.+'
            result = re.findall(pattern, s)
            results[context] = cpc_result + ". " + result[0].lstrip(pattern)
    return results


cpc_texts = get_cpc_texts()
#torch.save(cpc_texts, OUTPUT_DIR+"cpc_texts.pth")
#df_train['context_text'] = df_train['context'].map(cpc_texts)
#df_test['context_text'] = df_test['context'].map(cpc_texts)
#display(df_train.head())
#display(df_test.head())

In [ ]:
cpc_texts['A01']

AGRICULTUREのAが消えてしまう。違う単語になっているので、気になる。

In [ ]:
# ====================================================
# CPC Data
# ====================================================
def get_cpc_texts():
    contexts = []
    pattern = '[A-Z]\d+'
    for file_name in os.listdir('../input/cpc-data/CPCSchemeXML202105'):
        result = re.findall(pattern, file_name)
        if result:
            contexts.append(result)
    contexts = sorted(set(sum(contexts, [])))
    contexts2 = []
    pattern = '[A-Z]\d+[A-Z]'
    for file_name in os.listdir('../input/cpc-data/CPCSchemeXML202105'):
        result = re.findall(pattern, file_name)
        if result:
            contexts2.append(result)
    contexts2 = sorted(set(sum(contexts2, [])))
    results = {}
    for cpc in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'Y']:
        df_cpc=pd.read_csv(f'../input/cpc-data/CPCTitleList202202/cpc-section-{cpc}_20220201.txt', sep='\t',names=('seg','num','context'))
        cpc_result=df_cpc[df_cpc['seg']==cpc]['context'].values
#        print(cpc_result)
        for context in [c for c in contexts if c[0] == cpc]:
            result = df_cpc[df_cpc['seg']==context]['context'].values
            con_text=str.lower(cpc_result[0]) + ". " + str.lower(result[0])
            results[context] = con_text
    return results,contexts,contexts2


cpc_texts,contexts,contexts2 = get_cpc_texts()
df_train['context_text'] = df_train['context'].map(cpc_texts)
df_test['context_text'] = df_test['context'].map(cpc_texts)
display(df_train.head())
display(df_test.head())

In [ ]:
cpc_texts['A01']

agricultureがちゃんと認識されて、LBが0.833から0.8365に上がりました。  
trainのnotebookはpublicにしていましたが、privateにされてます。

cpc-contextを1000文字まで大きくしてみたが、訓練時間が増加するだけで、LBは上がりませんでした。